# A Mini Project Analyzing School Attendance Rates from 2021-2022

## By Lindsay Toothaker and Savannah Dorey

### Data Source

The dataset is downloaded from [Data.gov](https://catalog.data.gov/dataset/school-attendance-by-student-group-and-district-2021-2022). The data was collected from 2021-2022 and updated in 2023. It includes attendance rate data on U.S. public school students by district and student group from PK-12. We intend on answer questions regarding which district and category of students have the highest and lowest attendance rates and if/how 


In [1]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd

import matplotlib.pyplot as plt

df = pd.read_csv('s3://ia340-123457/School_Attendance_by_Student_Group_and_District__2021-2022.csv') # load the data from s3
df.head() #show the top 5 rows

Matplotlib is building the font cache; this may take a moment.


PermissionError: Forbidden

### Data Exploration and Cleaning

In [ ]:
df.shape #show the number of rows and columns 

In [ ]:
df.sort_values(by='Student group') #sort data by a column

In [ ]:
df.info() #show the data types and number of null values 

In [ ]:
#rename columns
df= df.rename(columns={"Category": "Need Type", "Student Group": "Student Demographic Group" }) 
df.head()

In [ ]:
df.isnull().sum() #show the number of rows containing null value

In [ ]:
df = df.dropna() #drop null values
df.isnull().sum()

In [ ]:
df.describe() #statistical summary of the data

In [ ]:
df.to_csv('clean_school_data.csv') #store the cleaned data to local computer

In [ ]:
#df.hist(layout=(2,2), figsize=(10, 5), sharey=False, sharex=False, bins=50) #use multiple charts to show the distribution of all columns

### Data Visualization

In [ ]:
avg_attendance_by_need_type = df.groupby('Need Type')['2020-2021 attendance rate'].mean() #aggregate the attendance rate per need type
avg_attendance_by_need_type.head()

In [ ]:
avg_attendance_by_need_type.plot.bar()

In [ ]:
avg_attendance_by_district_name = df.groupby('District name')['2020-2021 attendance rate'].mean() #aggregate the attendance rate per district name
avg_attendance_by_district_name.head()

In [ ]:
avg_attendance_by_district_name.head().plot()

In [ ]:
mean_att_rate_by_district = df.groupby('District name')['2020-2021 attendance rate'].mean() # calculate the average attendance rates per district
mean_att_rate_by_district.nsmallest(10).plot.bar() # use a bar chart to show the results of the smallest 10 districts 

In [ ]:
#df.plot.scatter(x='district name',y='2020-2021 attendance rate') #use scatter plot to show the result

### Utilizing AI

In [ ]:
pip install jupyter-ai~=1.0

In [ ]:
pip install jupyter-ai[all]

In [ ]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)


In [ ]:
import psycopg2

conn = psycopg2.connect(host=get_secret('postgresql')['host'],
                      user=get_secret('postgresql')['username'],
                      password = get_secret('postgresql')['password'],
                      dbname=get_secret('postgresql')['engine'])

cur = conn.cursor()

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = get_secret('openai')['api_key']

In [ ]:
%load_ext jupyter_ai_magics

In [ ]:
%ai list

In [ ]:
%%ai gpt4
What is in the {df} in {In[10]}

### Question 1

In [ ]:
%%ai gpt4 -f code

Calculate which district had the lowest attendance rate from 2020-2021? 
Which district had the highest?

what are the results?

In [ ]:
# Find the district with the lowest attendance rate in 2020-2021
lowest_attendance_district = df[df['2020-2021 attendance rate'] == df['2020-2021 attendance rate'].min()]['District name'].iloc[0]

# Find the district with the highest attendance rate in 2020-2021
highest_attendance_district = df[df['2020-2021 attendance rate'] == df['2020-2021 attendance rate'].max()]['District name'].iloc[0]

print("District with lowest attendance rate (2020-2021): ", lowest_attendance_district)
print("District with highest attendance rate (2020-2021): ", highest_attendance_district)

### Question 2

In [ ]:
%%ai gpt4 -f code

Calculate which need type had the lowest attendance rate from 2020-2021?

What are the results?

In [ ]:
# Assuming 'df' is your DataFrame and it has columns 'Need Type' and '2020-2021 attendance rate'

# Group by need type and calculate mean attendance rate
grouped_df = df.groupby('Need Type')['2020-2021 attendance rate'].mean()

# Find the need type with the lowest attendance rate
lowest_need_type = grouped_df.idxmin()
lowest_rate = grouped_df.min()

lowest_need_type, lowest_rate

### Question 3

In [ ]:
%%ai gpt4 -f code

Does need type have a significant impact on attendance rate? How so?

Do not use numpy

Can you show the results in a table

In [ ]:
# Assuming 'df' is your DataFrame and it has columns '2020-2021 attendance rate' and 'Need Type'
# We can use a t-test from the scipy library to determine if 'Need Type' has a significant impact on '2020-2021 attendance rate'.

import scipy.stats as stats

# Group the data by 'Need Type'
grouped = df.groupby('Need Type')

# Create lists to store the 'Need Type' and corresponding t-statistics and p-values
need_types = []
t_stats = []
p_values = []

# Conduct a t-test for each group against the entire '2020-2021 attendance rate' population
# Store the t-statistics and p-values in the respective lists
for name, group in grouped:
    t_stat, p_value = stats.ttest_ind(df['2020-2021 attendance rate'], group['2020-2021 attendance rate'])
    need_types.append(name)
    t_stats.append(t_stat)
    p_values.append(p_value)

# Create a DataFrame to store 'Need Type', 'T-statistic', and 'P-value'
df_results = pd.DataFrame({'Need Type': need_types, 'T-statistic': t_stats, 'P-value': p_values})

# Display the DataFrame
print(df_results)

### Question 4

In [ ]:
%%ai gpt4 -f code

Can you create a scatterplot with the need type and 2020-2021 attendance rate

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'df' is your DataFrame and it has columns 'Need Type', '2020-2021 Attendance Rate'
plt.scatter(df['Need Type'], df['2020-2021 attendance rate'])
plt.title('Scatterplot of Need Type vs 2020-2021 attendance rate')
plt.xlabel('Need Type')
plt.ylabel('2020-2021 attendance rate')
plt.grid(True)
plt.show()


### Question 5

In [ ]:
%%ai gpt4 -f code

Can you create a scatterplot with the 2020-2021 student count and 2020-2021 attendance rate and district name

Do not provide key for the scatterplot

Do not use numpy

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'df' is your DataFrame and it has columns '2020-2021 Student Count', '2020-2021 Attendance Rate' and 'District Name'
districts = df['District name'].unique()

for district in districts:
    district_data = df[df['District name'] == district]
    plt.scatter(district_data['2020-2021 student count'], district_data['2020-2021 attendance rate'])

plt.title('Scatterplot of 2020-2021 Student Count vs 2020-2021 Attendance Rate')
plt.xlabel('2020-2021 Student Count')
plt.ylabel('2020-2021 Attendance Rate')
plt.show()

# AI Analysis Interpretations

### The first question posed to AI was what school district has the highest vs. lowest attendance rate. OpenAI used min() and max() functions in the equations that it generated for calculating these. EdAdvance had the lowest attendance rate and Deep River School District had the highest attendance rate

### The second question asked what student need type had the lowest attendance rate. AI grouped the data by need type and then found the lowest mean, showing that Homelessness was the student need that resulted in the lowest attendance rate.

### Since need type seemed to influence attendance rates, we asked AI if that was statistically significant. It tried to use numpy to do this, but then we reworded the prompt to do this differently. It produced P-Values for each need type

### We also used need type as a variable to create a scatterplot showing attendance rate. This visualization showed the stark difference between attendance rates of students experiencing homelessness vs. students with different needs.

### A different scatterplot was created to address student count's relationship with attendance rate and the school district name, and while one was produced, it does not provide very helpful information for analyzing data trends. 